In [13]:
# install the dependencies
!pip install requests python-dotenv

In [14]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

session = requests.Session()

In [15]:
WIKIDATA_API_URL = 'https://www.wikidata.org/w/api.php'

# sandbox item
item = 'Q15397819'

In [16]:
res = session.get(WIKIDATA_API_URL, params={
    'action': 'wbgetentities',
    'ids': item,
    'format': 'json',
})
result = res.json()
result

{'entities': {'Q15397819': {'pageid': 17254039,
   'ns': 0,
   'title': 'Q15397819',
   'lastrevid': 1043451618,
   'modified': '2019-11-02T16:11:05Z',
   'type': 'item',
   'id': 'Q15397819',
   'labels': {'ko': {'language': 'ko', 'value': '세 번째 연습장'},
    'en': {'language': 'en', 'value': 'Wikidata Sandbox 3'},
    'en-ca': {'language': 'en-ca', 'value': 'Third sandbox'},
    'el': {'language': 'el', 'value': 'Τρίτο Πρόχειρο'},
    'ru': {'language': 'ru', 'value': 'Песочница: Эпизод III'},
    'zh-hans': {'language': 'zh-hans', 'value': '维基数据沙盒3'},
    'mk': {'language': 'mk', 'value': 'Трет песочник'},
    'eo': {'language': 'eo', 'value': 'tria provejo'},
    'nl': {'language': 'nl', 'value': 'derde zandbak'},
    'cs': {'language': 'cs', 'value': 'třetí pískoviště'},
    'it': {'language': 'it', 'value': 'terza pagina delle prove'},
    'sv': {'language': 'sv', 'value': 'Tredje sandlådan'},
    'ja': {'language': 'ja', 'value': 'ウィキデータ・サンドボックス3'},
    'fr': {'language': 'fr', 'va

In [17]:
# login to wikidata
# Note: the login is saved in the requests session (i.e. in the cookies)
#       so make sure to use the same session for all subsequent calls
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'login',
    'format': 'json',
})
tokens = res.json()['query']['tokens']

res = session.post(WIKIDATA_API_URL, data={
    'action': 'login',
    'lgname': os.environ['BOT'],
    'lgpassword': os.environ['BOT_PW'],
    'lgtoken': tokens['logintoken'],
    'format': 'json'
})
login = res.json()
login

{'login': {'result': 'Success', 'lguserid': 733408, 'lgusername': 'Metaodi'}}

In [18]:
# generate csrf token
res = session.get(WIKIDATA_API_URL, params={
    'action': 'query',
    'meta': 'tokens',
    'type': 'csrf',
    'format': 'json',
})
csrf = res.json()['query']['tokens']['csrftoken']
csrf

# update description of sandbox item
res = session.post(WIKIDATA_API_URL, data={
    'action': 'wbsetdescription',
    'id': item,
    'token': csrf,
    'language': 'en',
    'value': 'My super wikidata bot',
    'format': 'json',
})
res.json()

{'entity': {'descriptions': {'en': {'language': 'en',
    'value': 'My super wikidata bot'}},
  'id': 'Q15397819',
  'type': 'item',
  'lastrevid': 1043453079},
 'success': 1}

In [25]:
# request the item again
res = session.get(WIKIDATA_API_URL, params={
    'action': 'wbgetentities',
    'ids': item,
    'format': 'json',
})
result = res.json()
result['entities']['Q15397819']['descriptions']['en']

{'language': 'en', 'value': 'My super wikidata bot'}